## Shortest Path



### Dijkstra's Algorithm

Prerequisite: **non-negative** weighted edges

DataStructures: 
- `dist[]`: distance from source to each vertex. Initialize to `infinity` for all vertices and `0` for the source.
- `edgeTo[]`: previous vertex on the shortest path. Initialize to `null` for all vertices.
- `MinPQ`: priority queue to get the vertex with the smallest distance. Enqueue the source with distance `0` before starting the algorithm.

Algorithm:
MinPQ != empty:
   1. Dequeue (get the vertex with the smallest distance) and relax its adjacent edges
   2. Compare the distance whether it's less than previous distance
      if less than:
      3. Update the distance and previous vertex
      4. Enqueue the vertex with the new distance (or decrease key before enqueuing)



In [14]:
from typing import TypeVar, Dict, List, Set
T =TypeVar("T")

class DirWeightedEdge:
    def __init__(self, src:T, dest:T, weight:int):
        self.src = src
        self.dest = dest
        self.weight = weight
    
    def __str__(self):
        return f"{self.src} -> {self.dest} | {self.weight}"
    
    def __repr__(self):
        return self.__str__()
    

class DirWeightedGraph:
    def __init__(self, start:T, graph:Dict[T, List[DirWeightedEdge]] = {}):
        self.start = start
        self.graph: Dict[T, List[DirWeightedEdge]] = graph
        self.vertices: Set[T] = set()

    def add_edge(self, src:T, dest:T, weight:int):
        if src not in self.graph:
            self.graph[src] = []
        self.graph[src].append(DirWeightedEdge(src, dest, weight))

        self.vertices.add(src)
        self.vertices.add(dest)

    def get_vertices(self) -> List[T]:
        return list(self.vertices)
    
    def get_adj_edges(self, src:T) -> List[DirWeightedEdge]:
        return self.graph.get(src, [])
    
    def get_start(self) -> T:
        return self.start

In [15]:
import heapq
from typing import Tuple, List, Dict

class DijkstraSP:
    def __init__(self, graph:DirWeightedGraph):
        self.graph = graph

        self.distTo: Dict[T, int] = {}
        self.edgeTo: Dict[T, T] = {}

        self.minPQ: List[tuple[int, T]] = []
        
    def dijkstra(self, src:T):

        # Initialize the distance from every vertex to source as infinity(source to itself is 0), edgeTo as None
        self.distTo = {v : float('inf') for v in self.graph.get_vertices()}
        self.distTo[src] = 0
        self.edgeTo = {v : None for v in self.graph.get_vertices()}
        
        # Enqueue the source and its distance to source to PQ
        heapq.heappush(self.minPQ, (0, src))

        while self.minPQ:

            # 1. Dequeue the vertex with minimum dist 
            weight, curr_v  = heapq.heappop(self.minPQ)


            # Check if curr_v has already smaller distance
            if weight > self.distTo[curr_v]:
                continue

            # 2. Relax the edges from the current vertex
            for edge in self.graph.get_adj_edges(curr_v):
                weight = edge.weight
                dest_v = edge.dest
                new_dist = self.distTo[curr_v] + weight

                if (new_dist < self.distTo[dest_v]):

                    # 3. If the new distance is less than the older one, update the distance and edgeTo
                    self.distTo[dest_v] = new_dist
                    self.edgeTo[dest_v] = curr_v

                    # 4. enqueue the new distance to PQ
                    heapq.heappush(self.minPQ, (self.distTo[dest_v], dest_v))

    def get_shortest_paths(self) -> List[tuple[List[T],int]]: 
        self.dijkstra(self.graph.get_start())
        shortest_paths: List[Tuple[List[T], int]] = []

        # Get the shortest path from source to every vertex
        for dest in self.graph.get_vertices():
            path = []
            curr = dest
            while curr is not None: # (!=source)
                path.append(curr)
                curr = self.edgeTo[curr]
            shortest_paths.append((path[::-1], self.distTo[dest]))

        return shortest_paths

            


Dijkstra 

```python

def dijkstra(graph, source):
    distTo = {v : for v in graph.V()}
    distTo[source] = 0
    edgeTo = {v : None for v in graph.V()}

    pq = MinPQ()
    pq.enqueue(source, 0) # sorted by distance

    while not pq.empty():
        v = pq.dequeue() # vertex with the smallest distance
        for edge in graph.adj(v):
            relax(edge)

def relax(edge):
    v = edge.from()
    w = edge.to()
    if distTo[v] + edge.weight < distTo[w]:
        distTo[w] = distTo[v] + edge.weight
        edgeTo[w] = v
        if w in pq:
            # delete the old key and insert the new one
            pq.decreaseKey(w, distTo[w]) 
        else:
            pq.enqueue(w, distTo[w])

```python
def acyclicSP(graph, source):
    distTo = {v : for v in graph.V()}
    distTo[source] = 0
    edgeTo = {v : None for v in graph.V()}

    post_order = DFS(graph)
    topological_order = post_order.reverse()
    for v in topological_order:
        for edge in graph.adj(v):
            relax(edge)

### Edge-Weighted DAG Algorithm

Prerequisite: Directed Acyclic Graph (DAG), edge weights can be negative

1. Get the topological order of the DAG (DFS with post-order, and reverse the order)
2. Relax the edges and update the distances in topological order


In [16]:
class AcyclicSP:
    def __init__(self, graph: DirWeightedGraph):
        self.graph = graph

        self.distTo: Dict[T, int] = {}
        self.edgeTo: Dict[T, T] = {}

        self.topological_order: List[T] = []

    def dfs_post_order(self) -> List[T]:
        visited: Set[T] = set()
        res_post_order: List[T] = []

        def dfs(v):
            visited.add(v)
            for adj in self.graph.get_adj_edges(v):
                if adj.dest not in visited:
                    dfs(adj.dest)
        
            res_post_order.append(v)

        for v in self.graph.get_vertices():
            if v not in visited:
                dfs(v)

        return res_post_order

    def acyclic_sp(self,source):

        self.distTo = {v: float('inf') for v in self.graph.get_vertices()}
        self.distTo[source] = 0

        self.edgeTo = {v: None for v in self.graph.get_vertices()}

        # Get the topological order by reversing the post-order DFS
        self.topological_order = list(reversed(self.dfs_post_order))

        # Traverse the vertices in topological order
        for curr_v in self.topological_order:
            # Relax the edges from the current vertex
            for edge in self.graph.get_adj_edges(curr_v):
                weight = edge.weight
                dest_v = edge.dest
                new_dist = self.distTo[curr_v] + weight

                if new_dist < self.distTo[dest_v]:
                    self.distTo[dest_v] = new_dist
                    self.edgeTo[dest_v] = curr_v
                    


### Bellman-Ford Algorithm
Prerequisite: 
- Negative weight OK
- Cycles OK 
- No negative cycles !!!

Implementation:
- Traverse all edges `V-1` times.
- Stop if no distance is updated
- Traverse once more time to check for negative cycles.

```python
def bellman_ford(graph):

    distTo = {v : float('int') for v in graph.V()}
    distTo[source] = 0

    precursor = {v : None for v in graph.V()} # edgeTo[]

    for _ in range(graph.V() - 1):
        updated = False # flag to check if any distance is updated in this iteration

        for e in graph.edges():
            v = e.from()
            w = e.to()
            if distTo[v] + e.weight() < distTo[w]:
                distTo[w] = distTo[v] + e.weight()
                precursor[w] = v

                updated = True

        if not updated:
            break

    # check for negative cycles
    for e in graph.edges():
        v = e.from()
        w = e.to()
        if distTo[v] + e.weight() < distTo[w]:
            raise Exception("Negative cycle detected")  
```


In [17]:
def test_dijkstra():


    graph = DirWeightedGraph("S")
    graph.add_edge("S", "A", 5)
    graph.add_edge("S", "D", 9)
    graph.add_edge("S", "G", 8)
    graph.add_edge("A", "C", 15)
    graph.add_edge("A", "B", 12)
    graph.add_edge("A", "G", 4)
    graph.add_edge("B", "C", 3)
    graph.add_edge("B", "F", 11)
    graph.add_edge("C", "F", 9)
    graph.add_edge("D", "E", 4)
    graph.add_edge("D", "G", 5)
    graph.add_edge("D", "F", 20)
    graph.add_edge("E", "B", 1)
    graph.add_edge("E", "F", 13)
    graph.add_edge("G", "B", 7)
    graph.add_edge("G", "E", 6)

    
    dijkstra = DijkstraSP(graph)
    paths = dijkstra.get_shortest_paths()
    print("Shortest paths from", dijkstra.graph.get_vertices()[0], ":")
    for path, dist in paths:
        print(f"To {path[-1]}: Path = {path}, Distance = {dist}")


test_dijkstra()

Shortest paths from D :
To D: Path = ['S', 'D'], Distance = 9
To A: Path = ['S', 'A'], Distance = 5
To E: Path = ['S', 'D', 'E'], Distance = 13
To C: Path = ['S', 'D', 'E', 'B', 'C'], Distance = 17
To G: Path = ['S', 'G'], Distance = 8
To F: Path = ['S', 'D', 'E', 'B', 'F'], Distance = 25
To S: Path = ['S'], Distance = 0
To B: Path = ['S', 'D', 'E', 'B'], Distance = 14
